In [1]:
import pandas as pd
import numpy as np


In [2]:
cards_data=pd.read_csv("cards_data.csv")
cards_data.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [3]:
evaluation_features=pd.read_csv("evaluation_features.csv")
evaluation_features.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,23042130,2019-06-01 14:41:00,900,3412,$118.61,Swipe Transaction,61195,Panama City,FL,32401.0,5541,NaN
1,22702314,2019-03-21 13:45:00,1741,3025,$-204.00,Chip Transaction,49637,Tucson,AZ,85736.0,3509,NaN
2,22838524,2019-04-19 11:53:00,1081,3892,$-99.00,Chip Transaction,43293,Aurora,CO,80013.0,5499,NaN
3,23477219,2019-09-01 11:46:00,1740,2382,$28.44,Chip Transaction,26032,Mukwonago,WI,53149.0,7210,NaN
4,23066897,2019-06-06 21:24:00,1207,6056,$32.02,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN


In [4]:
transactions_train=pd.read_csv("transactions_train.csv")
transactions_train.head()

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,NaN
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,NaN
2,17566794,2016-03-26 12:42:00,1967,3367,$11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,5411,NaN
3,17318690,2016-02-01 08:30:00,921,3457,$85.74,Chip Transaction,63701,Rush,NY,14543.0,5411,NaN
4,20994060,2018-03-24 14:42:00,456,2800,$13.43,Chip Transaction,83271,Estero,FL,33928.0,4214,NaN


In [5]:
users_data=pd.read_csv("users_data.csv")
users_data.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [18]:
m1_train = pd.merge(
    transactions_train,
    users_data,
    how='left',             # type de jointure
    left_on='client_id',    # clé de jointure dans transaction_data
    right_on='id'           # clé de jointure dans client_data
)

m2_train = pd.merge(
    m1_train,
    cards_data,
    how='left',             
    left_on='card_id',    
    right_on='id'           
)

X_m1=m2_train

In [19]:
X_m1.columns

Index(['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'use_chip', 'merchant_id', 'merchant_city', 'merchant_state', 'zip',
       'mcc', 'errors', 'id_x', 'current_age', 'retirement_age', 'birth_year',
       'birth_month', 'gender', 'address', 'latitude', 'longitude',
       'per_capita_income', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards', 'id_y', 'client_id_y', 'card_brand', 'card_type',
       'card_number', 'expires', 'cvv', 'has_chip', 'num_cards_issued',
       'credit_limit', 'acct_open_date', 'year_pin_last_changed',
       'card_on_dark_web'],
      dtype='object')

In [20]:
y_dic=pd.read_json("train_fraud_labels.json")
y_m1 = y_dic.loc[X_train['transaction_id'], 'target']
y_m1.head()

19765990    No
22160255    No
17566794    No
17318690    No
20994060    No
Name: target, dtype: object

In [21]:
cols_to_drop = [
    'transaction_id', 'client_id_x', 'client_id_y', 'id_x', 'id_y',
    'card_number', 'expires', 'cvv', 'address', 'acct_open_date', 'date'
]

X_clean = X_m1.drop(columns=cols_to_drop)

y_clean = y_m1.values

In [29]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42)
#model.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_rapide = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
}

param_grid_lent = {
    'n_estimators': [100, 200, 300, 500],        # + d'arbres = + de stabilité
    'max_depth': [10, 20, 30, None],             # autorise des arbres plus complexes
    'min_samples_split': [2, 5, 10],             # plus de granularité sur les divisions
    'min_samples_leaf': [1, 2, 4],               # évite les feuilles trop petites
    'max_features': ['sqrt', 'log2', None],      # explore plus d'options
    'bootstrap': [True, False],                  # True = bagging classique
    'class_weight': ['balanced', None],          # utile si tes classes sont déséquilibrées
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,  # 3
    scoring='f1',  # 'precision', 'recall', 'roc_auc'
    n_jobs=-1,  
    verbose=2
)

#grid_search.fit(X_encoded, y_train)
#best_model = grid_search.best_estimator_